In [ ]:
from utils import *

#### GPU setting

In [ ]:
# Sets limit on and selects index-th gpu
gpus = tf.config.list_physical_devices("GPU")
index = 0

tf.config.experimental.set_memory_growth(gpus[index], True)
# select the gpu you want to use
if gpus:
    # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.set_visible_devices(gpus[index], "GPU")
        logical_gpus = tf.config.list_logical_devices("GPU")
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)

### Data Preprocessing, Augmenting,  Loading and checking shape

In [ ]:
data_loader = DataLoader(root_path="Data", is_aug=True, sample_count=10)
train_data, test_data = data_loader()
_, train_target, train_labels = train_data

# ------- load test data from folders not from .npy -------
# _, test_target, test_labels = data_loader.DataLoad(data_path="Data/benchmarking_intive/", decode_target=True)
_, test_target, test_labels = data_loader.DataLoad(
    data_path="Data/test/", decode_target=False
)
# ------- load test data from folders not from .npy -------

### Creating model and checking summary

In [ ]:
split_len = 50
model = keras.Sequential(
    [
        tf.keras.Input(shape=(split_len - 2, 8), batch_size=None),
        tf.keras.layers.GRU(units=64, return_sequences=False),
        tf.keras.layers.Dense(2),
        tf.keras.layers.Softmax(),
    ]
)

weights_path = "models/test.h5"
model.save_weights(weights_path)
model.summary()

### Trainer Function

In [ ]:
def trainer(
    model, data, weights_path, data_test, batch_size=64, epochs=15, learning_rate=0.003
):
    X_train, Y_train = data

    # Declaring training checkpoints (save weights --> best only)
    checkpoint = keras.callbacks.ModelCheckpoint(
        weights_path, monitor="val_loss", verbose=0, save_best_only=True, mode="min"
    )
    callbacks_list = [
        checkpoint
    ]  # , tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)]

    # Start training,
    history = model.fit(
        X_train,
        Y_train,
        batch_size=batch_size,
        epochs=epochs,
        verbose=1,
        callbacks=callbacks_list,
        validation_data=data_test,
    )

    return history

### Start Training

In [ ]:
learning_rate = 0.001

kicks = train_target[np.where(train_labels == 1)[0]]
no_kicks = train_target[np.where(train_labels == 0)[0]]

# this is for balanced training
no_kick_splits = np.array_split(
    no_kicks,
    len(no_kicks) // len(kicks) + 1
    if len(no_kicks) % len(kicks)
    else len(no_kicks) / len(kicks),
)

# Compiling model, declaring Loss function (Sparse Categorical Crossentropy), optimizer (ADAM)
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["acc"],
    optimizer=keras.optimizers.Adam(
        learning_rate=learning_rate,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-07,
        decay=0,
        amsgrad=False,
    ),
)

for no_kick_i in no_kick_splits:
    train_data_i = np.concatenate([no_kick_i, kicks], axis=0)
    train_labels_i = np.concatenate([np.zeros(len(no_kick_i)), np.ones(len(kicks))])
    history = trainer(
        model=model,
        epochs=10,
        data=(train_data_i, train_labels_i),
        data_test=(test_target, test_labels),
        batch_size=64,
        weights_path=weights_path,
    )

### Checking Training Results

In [ ]:
# Model loss in per epoch
plt.plot(history.history["loss"])
plt.title("model loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["train", "test"], loc="upper left")
plt.show()

### Model evaluation on test (unseen, noaugmented) data

In [ ]:
# model.load_weights('models/our_train_our_fit_full.h5')
# _, test_data, test_labels = data_loader.DataLoad(data_path="Data/benchmarking_intive", decode_target=True)
# test_data, test_labels = data_loader.TestDataLoad()
_, test_data, test_labels = data_loader.DataLoad(
    data_path="Data/test/", decode_target=False
)

In [ ]:
test_data.shape, test_labels.shape

In [ ]:
test_labels.mean()

### <span style="color:blue">ACC</span>

In [ ]:
pred = model.predict(test_data, verbose=0)

# Calculate ACC
print("\033[1m TEST ACC -->", CalcACC(pred, test_labels), "%")

### <span style="color:blue">Kick ACC</span>

In [ ]:
pred_kick = model.predict(test_data[np.where(test_labels == 1)[0]], verbose=0)

# Calculate ACC
print(
    "\033[1m TEST KICK ACC -->",
    CalcACC(pred_kick, np.ones(len(test_data[np.where(test_labels == 1)[0]]))),
    "%",
)

### <span style="color:blue">No kick ACC</span>

In [ ]:
pred_no_kick = model.predict(test_data[np.where(test_labels == 0)[0]], verbose=0)

# Calculate ACC
print(
    "\033[1m TEST NO KICK ACC -->",
    CalcACC(pred_no_kick, np.zeros(len(test_data[np.where(test_labels == 0)[0]]))),
    "%",
)

### <span style="color:blue">Precision and recall</span>

In [ ]:
tn, fp, fn, tp = confusion_matrix(test_labels, pred.argmax(axis=-1)).ravel()
precision = tp / (tp + fp)
recall = tp / (tp + fn)
print("\033[1m TEST PRECISION -->", np.round(precision * 100, 2), "%")
print("\033[1m TEST RECALL -->", np.round(recall * 100, 2), "%")

|  ACC   | Kick Acc | No kick Acc  |   P   |   R   |
|:------:|:--------:|:------------:|:-----:|:-----:|
| 98.17  |  86.5    |    99.66     | 96.98 | 86.5  |